# Irina Viner-Usmanova Rhythmic Gymnastics Centre

In [1]:
using WebIO

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-18192369870551955147\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-12177898108449017880\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-18192369870551955147\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

In [2]:
using Interact

In [3]:
using Khepri

In [4]:
render_size(800, 400) # Khepri backends size

(800, 400)

Avoid tests macro: `true` to skip tests, and `false` to run them.

In [5]:
avoid_tests = Parameter(true)

macro test(expr...)
  quote
    if !avoid_tests() 
        begin
            $(esc(expr...))
        end
    end
  end
end

@test (macro with 1 method)

In [31]:
avoid_tests(false)

false

## Sinusoidal Curves

 | Sinunoisal wave parameters | Other sinusoidal curves |
 |----------------------------|-------------------------|
 |<img src="./Figures/sin_wave.png" width="480"> | <img src="./Figures/sin_curves.png" width="350">|

`sinusoidal` parameters explained:
* a is the amplitude
* omega is the frequency
* fi is the phase

In [7]:
sinusoidal(a, omega, fi, x) = a*sin(omega*x+fi)

sinusoidal (generic function with 1 method)

In [8]:
sin_array_y(p, a, omega, fi, dist, n) = [p+vxy(i, sinusoidal(a, omega, fi, i)) for i in division(0, dist, n)]

sin_array_y (generic function with 1 method)

In [9]:
@test begin
    backend(notebook)
    new_backend()
    line(sin_array_y(u0(), 5, 1, 0, 50, 100))
end

Expected result:
<img src="./Plots/sin_array_y.png" width="600">

`damped_sin_wave`'s parameters explained
* a is the initial amplitude (the highest peak)
* b is the decay constant
* omega is the angular frequency

In [10]:
damped_sin_wave(a, d, omega, x) = a*exp(-(d*x))*sin(omega*x)

damped_sin_wave (generic function with 1 method)

In [11]:
damped_sin_array_z(p, a, d, omega, dist, n) = [p+vxz(i, damped_sin_wave(a, d, omega, i)) for i in division(0, dist, n)]

damped_sin_array_z (generic function with 1 method)

In [12]:
@test begin
    backend(notebook)
    new_backend()
    line(damped_sin_array_z(u0(), 5, 0.1, 1, 50, 100))
end

Expected result:
<img src="./Plots/damped_sin_array.png" width="600">

In [13]:
damped_sin_roof_pts(p, a_x, a_y, fi, decay, om_x, om_y, dist_x, dist_y, n_x, n_y) =
    map_division((x, y)->p+vxyz(x, y, sinusoidal(a_x, om_x, fi, x)+damped_sin_wave(a_y, decay, om_y, y)),
        0, dist_x, n_x,
        0, dist_y, n_y)

damped_sin_roof_pts (generic function with 1 method)

In [14]:
@test begin
    backend(meshcat)
    new_backend()
    @manipulate for ampx = widget(0:0.1:3, label="Amplitude x"),
                ampy = widget(0:0.1:20, label="Amplitude y")
            delete_all_shapes()
            surface_grid(damped_sin_roof_pts(u0(), ampx, ampy, 0, 0.1, 0.2, 1, 50, 100, 100, 200))
    end
end

Expected result:
<img src="./Plots/damped_surface.png" width="600">

In [15]:
damped_sin_roof_pts(p, h, a_x, a_y_min, a_y_max, fi, decay, om_x, om_y, dist_x, dist_y, n_x, n_y) =
    map_division((x, y) ->
                y <= d_i ?
                p + vxyz(x, 
                         -sin(y/d_i*(1*pi)), 
                         y*h/d_i + sin(x/dist_x*pi)sinusoidal(a_x, om_x, fi-y*pi/dist_y, x)*(y*a_x/d_i)) :
                p + vxyz(x,
                         y,
                         h + sin(x/dist_x*pi)*sinusoidal(a_x, om_x, fi-y*pi/dist_y, x) +
                         damped_sin_wave(a_y_max - (a_y_max-a_y_min)/dist_x*x, decay, om_y, y)),
             0, dist_x, n_x,
             0, dist_y, n_y)

damped_sin_roof_pts (generic function with 2 methods)

`d_i`is the distance between the pavillion starting point and the beginning of the dumped sine curve.

In [50]:
@test begin
    backend(meshcat)
    new_backend()
    d_i = 0.5
    @manipulate for ampx = widget(1:0.1:3, label="Amplitude x"),
                    ampy_min = widget(1:0.1:15, label="Minimum Amplitude y"),
                    ampy_max = widget(1:0.1:15, label="Maximum Amplitude y"),
                    h = widget(2:1:20, label="Height")
                delete_all_shapes()
                surface_grid(damped_sin_roof_pts(u0(), h, ampx, ampy_min, ampy_max, pi, 0.03, pi/50, pi/10, 60, 100, 120, 800))
    end
end

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://localhost:8700
└ @ MeshCat C:\Users\Renata\.julia\packages\MeshCat\ECbzr\src\visualizer.jl:73


Khepri.MCATViewer(MeshCat Visualizer with path /meshcat at http://localhost:8700)

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["Amplitude x"], Dict{Symbol,Any}(:className => "interact ",:style => Dict{Any,Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol,Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(:max => 21,:min => 1,:attributes => Dict{Any,Any}(:type => "range",Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient" => "horizontal"),:step => 1,:className => "slider slider is-fullwidth",:style => Dict{Any,Any}()))], Dict{Symbol,Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol,Any}(:className => "interact-flex-row-right"))], Dict{Symbol,Any}(:className => "interact-flex-row interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing),"index" => (Observable{Any} with 2 listeners. Value:
11, nothing)), Set(String[]), nothing, Asset[Asset("js", "knockout", "C:\\Users\\Renata\\.julia\\packages\\Knockout\\IP1uR\\src\\..\\assets\\knockout.js"), Asset("js", "knockout_punches", "C:\\Users\\Renata\\.julia\\packages\\Knockout\\IP1uR\\src\\..\\assets\\knockout_punches.js"), Asset("js", nothing, "C:\\Users\\Renata\\.julia\\packages\\InteractBase\\sOe2Z\\src\\..\\assets\\all.js"), Asset("css", nothing, "C:\\Users\\Renata\\.julia\\packages\\InteractBase\\sOe2Z\\src\\..\\assets\\style.css"), Asset("css", nothing, "C:\\Users\\Renata\\.julia\\packages\\Interact\\SbgIk\\src\\..\\assets\\bulma_confined.min.css")], Dict{Any,Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set(AbstractConnection[]), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x000000003f733990, Task (runnable) @0x000000003f733990), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"1.0\",\"1.1\",\"1.2\",\"1.3\",\"1.4\",\"1.5\",\"1.6\",\"1.7\",\"1.8\",\"1.9\",\"2.0\",\"2.1\",\"2.2\",\"2.3\",\"2.4\",\"2.5\",\"2.6\",\"2.7\",\"2.8\",\"2.9\",\"3.0\"],\"chan

Expected result:
<img src="./Plots/roof_surface.png" width="600">

## Pavilion Dimensions

 | Roof dimentions explained |
 |----------------------------|
 |<img src="./Figures/roof_dims.jpg" width="600"> |

__Double sinuoid parameters:__

* amp_x = amplitude of the sinusoid along the x axis
* amp_y_min_top = minimum amplitude of the damped sinusoid along the y axis for the top layer of the roof
* amp_y_max_top = maximum amplitude of the damped sinusoid along the y axis for the top layer of the roof
* amp_y_max_bottom = maximum amplitude of the damped sinusoid along the y axis for the bottom layer of the roof
* amp_y_min_bottom = minimum amplitude of the damped sinusoid along the y axis for the bottom layer of the roof
* fi = sinusoid's phase along the x axis
* decay = damped sinusoid's decay along the y axis
* om_x = frequency of the sinusoid in x
* om_y = frequency of the damped sinusoid in y

__Genaral pavilion parameters:__

* pav_width = pavilion's width (along x axis)
* pav_length = pavilion's lenght (along y axis)
* pav_height = pavilion's height (z axis)
* d_width = distance between roof layers in the x axis (top layer is wider)
* d_length = distance between roof layers in the y axis (top layer is longer)
* d_height = distance between roof layers in the z axis (height of the roof truss)
* d_i = ??

In [34]:
amp_x = 2.5
amp_y_min_top = 4
amp_y_max_top = 11
amp_y_max_bottom = 10
amp_y_min_bottom = 3
fi = pi
decay = 0.03
om_x = pi/50
om_y = pi/10
pav_width = 60
pav_length = 100
pav_height = 14
d_width = 1.5
d_length = 1
d_height = 3
d_i = 0.5

0.5

## Roof Truss

### Basic truss elements

The following functions create truss elements:
* `free_node` and `fixed_node`create truss nodes
* `bar`creates truss bars
* `panel`creates truss panels

CAD version of truss elements:

In [18]:
free_node(pt) = sphere(pt, 0.1)
fixed_node(pt) = sphere(pt, 0.1)

fixed_node (generic function with 1 method)

In [21]:
truss_panel(ps) = surface_polygon(ps)

truss_panel (generic function with 1 method)

BIM families for truss elements:

In [19]:
free_node_fam = truss_node_family_element(default_truss_node_family(), support=Khepri.truss_node_support(), radius=0.1)
sup_node_fam = truss_node_family_element(default_truss_node_family(), support=Khepri.truss_node_support(ux=true, uy=true, uz=true), radius=0.1)

TrussNodeFamily("truss_node_family", 0.1, Khepri.TrussNodeSupport(true, true, true, false, false, false, Parameter{Bool}(false)), TrussNodeFamily("truss_node_family", 0.2, false, nothing, IdDict{Backend,Khepri.Family}(Unity => Khepri.UnityMaterialFamily("Default/Materials/Steel"),AutoCAD => Khepri.ACADLayerFamily("TrussNodes", RGB{N0f8}(1.0,1.0,1.0), Parameter{Any}(nothing)),Khepri.POVRayBackend{Khepri.POVRayKey,Int64} => Khepri.BackendMaterialFamily{Khepri.POVRayMaterial}(Khepri.POVRayInclude("textures.inc", "texture", "Chrome_Metal")),Rhino => Khepri.RhinoLayerFamily("TrussNodes", RGB{N0f8}(1.0,1.0,1.0), Parameter{Any}(nothing)),Khepri.MCATBackend{Khepri.MCATKey,String} => Khepri.BackendMaterialFamily{NamedTuple{(:uuid, :type, :metalness, :roughness, :side, :color),Tuple{String,String,Int64,Float64,Int64,String}}}((uuid = "ff18db40-ccfe-11ea-320c-fd95d85c7b34", type = "MeshStandardMaterial", metalness = 1, roughness = 0.5, side = 2, color = "0xFFFFFF"))), IdDict{Backend,Any}()), IdDi

BIM version of truss elements:

In [20]:
fixed_node(p) = truss_node(p, family=sup_node_fam)
free_node(p) = truss_node(p, family=free_node_fam)

bar (generic function with 1 method)

In [ ]:
bar(p, q) = truss_bar(p, q)

### Spatial truss

`truss_ptss`receives two sets of points, the bottom and top points defining the shape of the truss to create.
It returns the points defining the truss structure.

In [22]:
truss_ptss(bottom_ptss,top_ptss) =
    length(top_ptss) == length(bottom_ptss) ?
    vcat([[pts,qts] for (pts,qts) in zip(bottom_ptss,top_ptss)]...) :
    vcat([[pts,qts] for (pts,qts) in zip(top_ptss[1:end-1],bottom_ptss)]..., [top_ptss[end]])

truss_ptss (generic function with 1 method)

`truss`receives the type of truss to create, as well as both bottom and top points defining its shape.
It creates a 3D spatial truss.

In [23]:
truss(truss_type; bottom_ptss=planar(p=x(0)), top_ptss=planar(p=z(1))) =
    let ptss = truss_ptss(bottom_ptss, top_ptss)
      truss_type(ptss)
  end

truss (generic function with 1 method)

 | Vierendeel modular block front | Vierendeel modular block side |
 |----------------------------|-------------------------|
 |<img src="./Figures/front_vierendeel.png" width="350"> | <img src="./Figures/side_vierendeel.png" width="350">|

`vierendeel`creates a vierendeel truss structure.

In [24]:
vierendeel(ptss, first=true) =
    let ais = ptss[1],
        bis = ptss[2],
        cis = ptss[3],
        dis = ptss[4]
      (first ? fixed_node : free_node).(ais)
      free_node.(bis)
      bar.(ais, bis)
      bar.(ais, cis)
      bar.(bis, dis)
      bar.(ais[2:end], ais[1:end-1])
      bar.(bis[2:end], bis[1:end-1])
      if ptss[5:end] == []
          fixed_node.(cis)
          free_node.(dis)
          bar.(cis[2:end], cis[1:end-1])
          bar.(dis[2:end], dis[1:end-1])
          bar.(dis, cis)
      else
          vierendeel(ptss[3:end], false)
      end
    end

vierendeel (generic function with 2 methods)

In [25]:
@test begin
    backend(meshcat)
    new_backend()
    @manipulate for ampx = widget(1:0.1:3, label="Amplitude x"),
                    ampy_top_min = widget(3:0.1:15, label="Minimum Amplitude y"),
                    ampy_top_max = widget(3:0.1:15, label="Maximum Amplitude y"),
                    ampy_bottom_min = widget(1:0.1:12, label="Minimum Amplitude y"),
                    ampy_bottom_max = widget(1:0.1:12, label="Maximum Amplitude y"),
                    h = widget(2:1:20, label="Height")
                delete_all_shapes()
        truss(vierendeel, 
          top_ptss=damped_sin_roof_pts(u0(), pav_height,
                                          ampx, ampy_top_min, ampy_top_max,
                                          fi, decay, om_x, om_y,
                                          pav_width, pav_length-d_length, 50, 80),
          bottom_ptss=damped_sin_roof_pts(xy(d_width,d_length*1), pav_height-d_height,
                                         ampx, ampy_bottom_min, ampy_bottom_max,
                                         fi, decay, om_x, om_y,
                                         pav_width - d_width*2, pav_length-2*d_length, 50, 80))
    end
end

Expected result:
<img src="./Plots/truss_roof_meshcat.png" width="800">

In [55]:
@test begin
    backend(unity)
    delete_all_shapes()
    ground()
    truss(vierendeel, 
      top_ptss=damped_sin_roof_pts(u0(), pav_height,
                                      amp_x, amp_y_min_top, amp_y_max_top,
                                      fi, decay, om_x, om_y,
                                      pav_width, pav_length-d_length, 5, 8),
      bottom_ptss=damped_sin_roof_pts(xy(d_width,d_length*1), pav_height-d_height,
                                     amp_x, amp_y_min_bottom, amp_y_max_bottom,
                                     fi, decay, om_x, om_y,
                                     pav_width - d_width*2, pav_length-2*d_length, 5, 8))
#     render_dir("C:\\Users\\Renata\\Documents\\GitHub\\GymnasticsCenter_Moscow\\Plots")
#     render_view("truss_roof_unity")
end

9-element Array{TrussBar,1}:
 TrussBar(...)
 TrussBar(...)
 TrussBar(...)
 TrussBar(...)
 TrussBar(...)
 TrussBar(...)
 TrussBar(...)
 TrussBar(...)
 TrussBar(...)

Expected result:
<img src="./Plots/truss_roof_unity.png" width="800">

## Roof Surface

Auxiliar functions

In [27]:
transpose_array(arrays) =
    [[row[i] for row in arrays]
     for i in 1:length(arrays[1])]

transpose_array (generic function with 1 method)

In [28]:
@test begin
    backend(meshcat)
    new_backend()
    @manipulate for ampx = widget(1:0.1:3, label="Amplitude x"),
                    ampy_top_min = widget(3:0.1:15, label="Minimum Amplitude y"),
                    ampy_top_max = widget(3:0.1:15, label="Maximum Amplitude y"),
                    ampy_bottom_min = widget(1:0.1:12, label="Minimum Amplitude y"),
                    ampy_bottom_max = widget(1:0.1:12, label="Maximum Amplitude y"),
                    h = widget(2:1:20, label="Height")
                delete_all_shapes()
        surface_grid(damped_sin_roof_pts(u0(), h,
                                   ampx, ampy_top_min, ampy_top_max,
                                   fi, decay, om_x, om_y,
                                   pav_width, pav_length-d_length, 50, 200))
        surface_grid(damped_sin_roof_pts(xy(d_width,d_length*1), h-d_height,
                                      ampx, ampy_bottom_min, ampy_bottom_max,
                                      fi, decay, om_x, om_y,
                                      pav_width - d_width*2, pav_length-2*d_length, 50, 200))
    end
end

In [28]:
@test begin
    backend(meshcat)
    new_backend()
    @manipulate for ampx = widget(1:0.1:3, label="Amplitude x"),
                ampy_top_min = widget(3:0.1:15, label="Minimum Amplitude y"),
                ampy_top_max = widget(3:0.1:15, label="Maximum Amplitude y"),
                ampy_bottom_min = widget(1:0.1:12, label="Minimum Amplitude y"),
                ampy_bottom_max = widget(1:0.1:12, label="Maximum Amplitude y"),
                h = widget(2:1:20, label="Height")
            delete_all_shapes()
            top_roof=damped_sin_roof_pts(x(0), h,
                                   ampx, ampy_top_min, ampy_top_max,
                                   fi, decay, om_x, om_y,
                                   pav_width, pav_length-d_length, 50, 100)
            bottom_roof=damped_sin_roof_pts(xy(d_width,d_length*1), h-d_height,
                                      ampx, ampy_bottom_min, ampy_bottom_max,
                                      fi, decay, om_x, om_y,
                                      pav_width - d_width*2, pav_length-2*d_length, 50, 100)
            surface_grid(top_roof)
            surface_grid(bottom_roof)
            loft_curves([spline(top_roof[1]),spline(bottom_roof[1])])
            loft_curves([line(top_roof[end]),line(bottom_roof[end])])
            loft_curves([line(transpose_array(top_roof)[end]),line(transpose_array(bottom_roof)[end])])
    end
end

## Glass Façade

Auxiliar functions:

In [28]:
itera_sweeked(ptss) =
    vcat([[let p23=intermediate_loc(p2, p3),
               p24=intermediate_loc(p2, p4)
             [p0,p1,p24,p23]
           end
          for (p0,p1,p2,p3,p4)
          in zip(pts0[1:end-1], pts1[1:end-1], pts1[2:end], pts0[2:end], pts2[2:end])]
         for (pts0, pts1, pts2)
         in zip(ptss[1:end-2], ptss[2:end-1],ptss[3:end])]...)

`damped_sin_glass_wall`creates the pavillion glass walls

In [ ]:
damped_sin_glass_wall(p, a_y, fi, decay, om_y, dist_y, dist_z, n_y, n_z) =
    map_division((y, z) ->
                    y <= d_i ?
                    p + vyz(-sin(y/d_i*(1*pi)), z) :
                    p + vyz(y,
                            z + (z/dist_z)*damped_sin_wave(a_y, decay, om_y, y)),
             0, dist_y, n_y,
             0, dist_z, n_z)

In [ ]:
@test begin
    backend(meshcat)
    new_backend()
    @manipulate for ampy_bottom_min = widget(1:0.1:12, label="Minimum Amplitude y"),
                    ampy_bottom_max = widget(1:0.1:12, label="Maximum Amplitude y"),
                    h = widget(2:1:20, label="Height")
                delete_all_shapes()
                west_glass_wall = damped_sin_glass_wall(x(d_width),
                                                        ampy_bottom_max,
                                                        fi, decay, om_y,
                                                        pav_length-2*d_length, h-d_height, 85, 20)
                east_glass_wall = damped_sin_glass_wall(x(pav_width-d_width),
                                                        ampy_bottom_min,
                                                        fi, decay, om_y,
                                                        pav_length-2*d_length, h-d_height, 85, 20)
                map(ps->surface_polygon(ps), itera_sweeked(west_glass_wall))
                map(ps->surface_polygon(ps), itera_sweeked(east_glass_wall))
    end
end

## Interior Walls

## Backend specifics

Ground:

In [44]:
ground_fam = slab_family_element(default_slab_family())
ground() =
    let x = 1000
        y = 1000
      slab(closed_polygonal_path([xy(-x,y), xy(x,y), xy(x,-y), xy(-x,-y)]), level(-0.05), ground_fam)
    end

ground (generic function with 2 methods)

BIM family materials for Unity backend:

In [47]:
set_backend_family(default_truss_bar_family(), unity, unity_material_family("Default/Materials/Aluminum"))
set_backend_family(free_node_fam, unity, unity_material_family("Default/Materials/Aluminum"))
set_backend_family(sup_node_fam, unity, unity_material_family("materials/metal/Copper"))
# set_backend_family(ground_fam, unity, unity_material_family("Default/Materials/White")) # -- white ground
set_backend_family(ground_fam, unity, unity_material_family("Default/Materials/WhiteUnlit")) # -- white ground no shadows

IdDict{Backend,Any} with 0 entries